In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

In [3]:
import sys
print(sys.executable)

c:\program files\python38\python.exe


In [4]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.master('local').appName('TFM').\
        config('spark.jars.packages',\
               'mysql:mysql-connector-java:8.0.27,com.datastax.spark:spark-cassandra-connector_2.12:3.3.0,org.mongodb.spark:mongo-spark-connector_2.12:3.0.1').\
        config("spark.sql.catalog.myCatalog", "com.datastax.spark.connector.datasource.CassandraCatalog").\
        config("spark.sql.catalog.client.spark.cassandra.connection.host", "127.0.0.1").\
        config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions").getOrCreate()

## Consulta 1

### Consulta 1 Cassandra

In [5]:
query = "SELECT COUNT(*) AS TotalRegistros FROM ( SELECT TP.EntityKey,\
    TP.StrategyPlanKey,e1.ScenarioName AS ScenarioName,e2.AccountName AS AccountName,e3.CurrencyName AS CurrencyName,\
    e4.ProductCategoryName AS ProductCategoryName,TP.Amount,TP.LoadDate\
    FROM FactStrategyPlan TP\
    LEFT JOIN dimscenario e1 ON TP.ScenarioKey = e1.ScenarioKey\
    LEFT JOIN dimaccount e2 ON TP.AccountKey = e2.AccountKey\
    LEFT JOIN dimcurrency e3 ON TP.CurrencyKey = e3.CurrencyKey\
    LEFT JOIN dimproductcategory e4 ON TP.ProductCategoryKey = e4.ProductCategoryKey) AS Subconsulta"
df_mysql = spark.read.format('jdbc').\
            option('url', 'jdbc:mysql://localhost:3306/spanish').\
            option('driver', 'com.mysql.jdbc.Driver').\
            option('user', 'root').\
            option('password', 'jepcpvmj1671E').\
            option('query', query).load()

In [6]:
total_registros = df_mysql.select("TotalRegistros").first()[0]
total_registros

2750628

Se da inico a la extracion de datos de MySQL y migracion a Cassandra

In [7]:
import time
# Se define el tamano de parte
tamaño_parte = 400000
offset = 0

# Inicia la toma de tiempo
inicio_tiempo = time.time()

# Realiza la extracción en partes, recorriendo hasta realizar la migracion completa
while offset < total_registros:
    query = f"SELECT TP.EntityKey,\
    TP.StrategyPlanKey,e1.ScenarioName AS ScenarioName,e2.AccountName AS AccountName,e3.CurrencyName AS CurrencyName,\
    e4.ProductCategoryName AS ProductCategoryName,TP.Amount,TP.LoadDate\
    FROM FactStrategyPlan TP\
    LEFT JOIN dimscenario e1 ON TP.ScenarioKey = e1.ScenarioKey\
    LEFT JOIN dimaccount e2 ON TP.AccountKey = e2.AccountKey\
    LEFT JOIN dimcurrency e3 ON TP.CurrencyKey = e3.CurrencyKey\
    LEFT JOIN dimproductcategory e4 ON TP.ProductCategoryKey = e4.ProductCategoryKey LIMIT {tamaño_parte} OFFSET {offset}"
    # Si es la última parte, ajusta el tamano de parte
    if offset + tamaño_parte >= total_registros:
        tamaño_parte = total_registros - offset
    df_mysql = spark.read.format('jdbc').\
            option('url', 'jdbc:mysql://localhost:3306/spanish').\
            option('driver', 'com.mysql.jdbc.Driver').\
            option('user', 'root').\
            option('password', 'jepcpvmj1671E').\
            option('query', query).load()
    df_mysql = df_mysql.withColumnRenamed('EntityKey', 'entitykey').withColumnRenamed('StrategyPlanKey', 'strategyplankey')\
    .withColumnRenamed('ScenarioName', 'scenarioname').withColumnRenamed('AccountName', 'accountname').withColumnRenamed('CurrencyName', 'currencyname')\
    .withColumnRenamed('ProductCategoryName', 'productcategoryname').withColumnRenamed('Amount', 'amount')\
    .withColumnRenamed('LoadDate', 'loaddate')
    df_mysql.write\
    .format("org.apache.spark.sql.cassandra")\
    .mode('append')\
    .options(table="tabla1", keyspace="tfm_jhoselinv")\
    .save()

    # Actualizar, para ir la siguiente parte
    offset += tamaño_parte
    print(offset)

# Detiene la toma de tiempo
fin_tiempo = time.time()

# Calcula el tiempo de migracion
tiempo_ejecucion = fin_tiempo - inicio_tiempo

print(f"Tiempo de ejecución: {tiempo_ejecucion} segundos")

400000
800000
1200000
1600000
2000000
2400000
2750628
Tiempo de ejecución: 54.67634057998657 segundos


Se indican los resgitros migrados a Cassandra consulta 1

In [8]:
df_cassandra = spark.read\
                .format("org.apache.spark.sql.cassandra")\
                .options(table="tabla1", keyspace="tfm_jhoselinv")\
                .load()
# Obtener el número de registros dela consulta realizada a Cassandra
num_rows = df_cassandra.count()
# Imprimir el número de registros
print("Número de registros en Cassandra:", num_rows)

Número de registros en Cassandra: 2750628


### Consulta 1 MongoBD

In [10]:
mongo_ip = "mongodb://localhost:27017/tfm_jhoselinv."
import time
tamaño_parte = 400000
offset = 0
# Inicia la toma de tiempo
inicio_tiempo = time.time()
# Realiza la extracción en partes, recorriendo hasta realizar la migracion completa
while offset < total_registros:
    query = f"SELECT TP.EntityKey,\
    TP.StrategyPlanKey,e1.ScenarioName AS ScenarioName,e2.AccountName AS AccountName,e3.CurrencyName AS CurrencyName,\
    e4.ProductCategoryName AS ProductCategoryName,TP.Amount,TP.LoadDate\
    FROM FactStrategyPlan TP\
    LEFT JOIN dimscenario e1 ON TP.ScenarioKey = e1.ScenarioKey\
    LEFT JOIN dimaccount e2 ON TP.AccountKey = e2.AccountKey\
    LEFT JOIN dimcurrency e3 ON TP.CurrencyKey = e3.CurrencyKey\
    LEFT JOIN dimproductcategory e4 ON TP.ProductCategoryKey = e4.ProductCategoryKey LIMIT {tamaño_parte} OFFSET {offset}"
    # Si es la última parte, ajusta el tamano de parte
    if offset + tamaño_parte >= total_registros:
        tamaño_parte = total_registros - offset
    df_mysql = spark.read.format('jdbc').\
            option('url', 'jdbc:mysql://localhost:3306/spanish').\
            option('driver', 'com.mysql.jdbc.Driver').\
            option('user', 'root').\
            option('password', 'jepcpvmj1671E').\
            option('query', query).load()
    df_mysql = df_mysql.withColumnRenamed('EntityKey', 'entitykey').withColumnRenamed('StrategyPlanKey', 'strategyplanKey')\
    .withColumnRenamed('ScenarioName', 'scenarioname').withColumnRenamed('AccountName', 'accountname').withColumnRenamed('CurrencyName', 'currencyname')\
    .withColumnRenamed('ProductCategoryName', 'productcategoryname').withColumnRenamed('Amount', 'amount')\
    .withColumnRenamed('LoadDate', 'loaddate')
    df_mysql.write\
    .format("com.mongodb.spark.sql.DefaultSource")\
    .option("uri", mongo_ip + "coleccion1")#coleccion1 es la coleccion que se creara en la base de datos de MongoDB
    .mode("append")\
    .save()

    # Actualizar el desplazamiento para la siguiente parte
    offset += tamaño_parte
    print(offset)

# Detiene la toma de tiempo
fin_tiempo = time.time()

# Calcula el tiempo de ejecución
tiempo_ejecucion = fin_tiempo - inicio_tiempo

print(f"Tiempo de ejecución: {tiempo_ejecucion} segundos")

SyntaxError: invalid syntax (2383929491.py, line 33)

In [11]:
mongo_ip = "mongodb://localhost:27017/tfm_jhoselinv."
num_documentos = spark.read.format("com.mongodb.spark.sql.DefaultSource").option('uri',mongo_ip+"coleccion1").load()
num_documentos = num_documentos.count()

# Imprimir el número de documentos
print("Número de documentos:", num_documentos)

Número de documentos: 0


# Consulta 2

### Consulta 2 en Cassandra

In [12]:
query = "SELECT COUNT(*) AS TotalRegistros FROM ( SELECT TP.channelKey,TP.SalesKey,e1.StoreName AS StoreName,e2.ProductName AS ProductName,\
    e3.PromotionName AS PromotionName,e4.CurrencyName AS CurrencyName,TP.UnitCost,TP.UnitPrice,TP.SalesQuantity,TP.ReturnQuantity,\
    TP.ReturnAmount,TP.DiscountQuantity,TP.DiscountAmount,TP.TotalCost,TP.SalesAmount,TP.LoadDate FROM factsales TP\
    LEFT JOIN dimstore e1 ON TP.StoreKey = e1.StoreKey \
    LEFT JOIN dimproduct e2 ON TP.ProductKey = e2.ProductKey\
    LEFT JOIN dimpromotion e3 ON TP.PromotionKey = e3.PromotionKey\
    LEFT JOIN dimcurrency e4 ON TP.CurrencyKey = e4.CurrencyKey) AS Subconsulta"
df_mysql = spark.read.format('jdbc').\
            option('url', 'jdbc:mysql://localhost:3306/spanish').\
            option('driver', 'com.mysql.jdbc.Driver').\
            option('user', 'root').\
            option('password', 'jepcpvmj1671E').\
            option('query', query).load()

In [13]:
total_registros = df_mysql.select("TotalRegistros").first()[0]
total_registros

3406089

In [14]:
import time
# Se define el tamano de parte
tamaño_parte = 400000
offset = 0

# Inicia la toma de tiempo
inicio_tiempo = time.time()

# Realiza la extracción en partes, recorriendo hasta realizar la migracion completa
while offset < total_registros:
    query = f"SELECT e0.ChannelName AS ChannelName,TP.SalesKey,e1.StoreName AS StoreName,e2.ProductName AS ProductName,\
    e3.PromotionName AS PromotionName,e4.CurrencyName AS CurrencyName,TP.UnitCost,TP.UnitPrice,TP.SalesQuantity,TP.ReturnQuantity,\
    TP.ReturnAmount,TP.DiscountQuantity,TP.DiscountAmount,TP.TotalCost,TP.SalesAmount,TP.LoadDate FROM factsales TP\
    LEFT JOIN dimchannel e0 ON TP.channelKey = e0.channelKey\
    LEFT JOIN dimstore e1 ON TP.StoreKey = e1.StoreKey \
    LEFT JOIN dimproduct e2 ON TP.ProductKey = e2.ProductKey\
    LEFT JOIN dimpromotion e3 ON TP.PromotionKey = e3.PromotionKey\
    LEFT JOIN dimcurrency e4 ON TP.CurrencyKey = e4.CurrencyKey LIMIT {tamaño_parte} OFFSET {offset}"
    # Si es la última parte, ajusta el tamano de parte
    if offset + tamaño_parte >= total_registros:
        tamaño_parte = total_registros - offset
    df_mysql = spark.read.format('jdbc').\
            option('url', 'jdbc:mysql://localhost:3306/spanish').\
            option('driver', 'com.mysql.jdbc.Driver').\
            option('user', 'root').\
            option('password', 'jepcpvmj1671E').\
            option('query', query).load()
    df_mysql = df_mysql.withColumnRenamed('ChannelName', 'channelname').withColumnRenamed('SalesKey', 'saleskey')\
    .withColumnRenamed('StoreName', 'storename').withColumnRenamed('ProductName', 'productname').withColumnRenamed('PromotionName', 'promotionname')\
    .withColumnRenamed('CurrencyName', 'currencyname').withColumnRenamed('UnitCost', 'unitcost')\
    .withColumnRenamed('UnitPrice', 'unitprice').withColumnRenamed('SalesQuantity', 'salesquantity')\
    .withColumnRenamed('ReturnQuantity', 'returnquantity').withColumnRenamed('ReturnAmount', 'returnamount')\
    .withColumnRenamed('DiscountQuantity', 'discountquantity').withColumnRenamed('DiscountAmount', 'discountamount')\
    .withColumnRenamed('TotalCost', 'totalcost').withColumnRenamed('SalesAmount', 'salesamount').withColumnRenamed('LoadDate', 'loaddate')
    df_mysql.write\
    .format("org.apache.spark.sql.cassandra")\
    .mode('append')\
    .options(table="tabla2", keyspace="tfm_jhoselinv")\
    .save()

    # Actualizar el desplazamiento para la siguiente parte
    offset += tamaño_parte
    print(offset)

# Detiene la toma de tiempo
fin_tiempo = time.time()

# Calcula el tiempo de ejecución
tiempo_ejecucion = fin_tiempo - inicio_tiempo

print(f"Tiempo de ejecución: {tiempo_ejecucion} segundos")

400000
800000
1200000
1600000
2000000
2400000
2800000
3200000
3406089
Tiempo de ejecución: 183.87224912643433 segundos


In [15]:
df_cassandra = spark.read\
                .format("org.apache.spark.sql.cassandra")\
                .options(table="tabla2", keyspace="tfm_jhoselinv")\
                .load()
# Obtener el número de registros dela consulta realizada a Cassandra
num_rows = df_cassandra.count()
# Imprimir el número de registros
print("Número de registros en Cassandra:", num_rows)

Número de registros en Cassandra: 3406089


### Consulta 2 MongoDB

In [16]:
mongo_ip = "mongodb://localhost:27017/tfm_jhoselinv."
import time
# Se define el tamano de parte
tamaño_parte = 400000
offset = 0

# Inicia la toma de tiempo
inicio_tiempo = time.time()

# Realiza la extracción en partes, recorriendo hasta realizar la migracion completa
while offset < total_registros:
    query = f"SELECT e0.ChannelName AS ChannelName,TP.SalesKey,e1.StoreName AS StoreName,e2.ProductName AS ProductName,\
    e3.PromotionName AS PromotionName,e4.CurrencyName AS CurrencyName,TP.UnitCost,TP.UnitPrice,TP.SalesQuantity,TP.ReturnQuantity,\
    TP.ReturnAmount,TP.DiscountQuantity,TP.DiscountAmount,TP.TotalCost,TP.SalesAmount,TP.LoadDate FROM factsales TP\
    LEFT JOIN dimchannel e0 ON TP.channelKey = e0.channelKey\
    LEFT JOIN dimstore e1 ON TP.StoreKey = e1.StoreKey \
    LEFT JOIN dimproduct e2 ON TP.ProductKey = e2.ProductKey\
    LEFT JOIN dimpromotion e3 ON TP.PromotionKey = e3.PromotionKey\
    LEFT JOIN dimcurrency e4 ON TP.CurrencyKey = e4.CurrencyKey LIMIT {tamaño_parte} OFFSET {offset}"
    # Si es la última parte, ajusta el tamano de parte
    if offset + tamaño_parte >= total_registros:
        tamaño_parte = total_registros - offset
    df_mysql = spark.read.format('jdbc').\
            option('url', 'jdbc:mysql://localhost:3306/spanish').\
            option('driver', 'com.mysql.jdbc.Driver').\
            option('user', 'root').\
            option('password', 'jepcpvmj1671E').\
            option('query', query).load()
    df_mysql = df_mysql.withColumnRenamed('ChannelName', 'channelname').withColumnRenamed('SalesKey', 'saleskey')\
    .withColumnRenamed('StoreName', 'storename').withColumnRenamed('ProductName', 'productname').withColumnRenamed('PromotionName', 'promotionname')\
    .withColumnRenamed('CurrencyName', 'currencyname').withColumnRenamed('UnitCost', 'unitcost')\
    .withColumnRenamed('UnitPrice', 'unitprice').withColumnRenamed('SalesQuantity', 'salesquantity')\
    .withColumnRenamed('ReturnQuantity', 'returnquantity').withColumnRenamed('ReturnAmount', 'returnamount')\
    .withColumnRenamed('DiscountQuantity', 'discountquantity').withColumnRenamed('DiscountAmount', 'discountamount')\
    .withColumnRenamed('TotalCost', 'totalcost').withColumnRenamed('SalesAmount', 'salesamount').withColumnRenamed('LoadDate', 'loaddate')
    df_mysql.write\
    .format("com.mongodb.spark.sql.DefaultSource")\
    .option("uri", mongo_ip + "coleccion2")\
    .mode("append")\
    .save()

    # Actualizar el desplazamiento para la siguiente parte
    offset += tamaño_parte
    print(offset)

# Detiene la toma de tiempo
fin_tiempo = time.time()

# Calcula el tiempo de ejecución
tiempo_ejecucion = fin_tiempo - inicio_tiempo

print(f"Tiempo de ejecución: {tiempo_ejecucion} segundos")

400000
800000
1200000
1600000
2000000
2400000
2800000
3200000
3406089
Tiempo de ejecución: 125.42827343940735 segundos


In [17]:
mongo_ip = "mongodb://localhost:27017/tfm_jhoselinv."
num_documentos = spark.read.format("com.mongodb.spark.sql.DefaultSource").option('uri',mongo_ip+"coleccion2").load()
num_documentos = num_documentos.count()

# Imprimir el número de documentos
print("Número de documentos:", num_documentos)

Número de documentos: 3406089


## Consulta 3

### Consulta 3 Cassandra

In [18]:
query = "SELECT COUNT(*) AS TotalRegistros FROM (SELECT e1.StoreType AS StoreType,TP.InventoryKey,e2.FullDateLabel AS FullDateLabel,\
        e3.ProductName AS ProductName,e4.CurrencyName AS CurrencyName,TP.OnHandQuantity, TP.OnOrderQuantity,\
        TP.SafetyStockQuantity, TP.UnitCost, TP.DaysInStock, TP.MaxDayInStock, TP.Aging FROM factinventory TP\
        LEFT JOIN dimstore e1 ON TP.StoreKey = e1.StoreKey\
        LEFT JOIN dimdate e2 ON TP.DateKey = e2.DateKey\
        LEFT JOIN dimproduct e3 ON TP.ProductKey = e3.ProductKey\
        LEFT JOIN dimcurrency e4 ON TP.CurrencyKey = e4.CurrencyKey) AS Subconsulta"
df_mysql = spark.read.format('jdbc').\
            option('url', 'jdbc:mysql://localhost:3306/spanish').\
            option('driver', 'com.mysql.jdbc.Driver').\
            option('user', 'root').\
            option('password', 'jepcpvmj1671E').\
            option('query', query).load()

In [19]:
total_registros = df_mysql.select("TotalRegistros").first()[0]
total_registros

8013099

In [20]:
import time
# Se define el tamano de parte 
tamaño_parte = 400000
offset = 0

# Inicia la toma de tiempo de tiempo
inicio_tiempo = time.time()

# Realiza la extracción en partes, recorriendo hasta realizar la migracion completa
while offset < total_registros:
    query = f"SELECT e1.StoreType AS StoreType,TP.InventoryKey,e2.FullDateLabel AS FullDateLabel,\
        e3.ProductName AS ProductName,e4.CurrencyName AS CurrencyName,TP.OnHandQuantity, TP.OnOrderQuantity,\
        TP.SafetyStockQuantity, TP.UnitCost, TP.DaysInStock, TP.MaxDayInStock, TP.Aging FROM factinventory TP\
        LEFT JOIN dimstore e1 ON TP.StoreKey = e1.StoreKey\
        LEFT JOIN dimdate e2 ON TP.DateKey = e2.DateKey\
        LEFT JOIN dimproduct e3 ON TP.ProductKey = e3.ProductKey\
        LEFT JOIN dimcurrency e4 ON TP.CurrencyKey = e4.CurrencyKey LIMIT {tamaño_parte} OFFSET {offset}"
    # Si es la última parte, ajusta el límite
    if offset + tamaño_parte >= total_registros:
        tamaño_parte = total_registros - offset
    df_mysql = spark.read.format('jdbc').\
            option('url', 'jdbc:mysql://localhost:3306/spanish').\
            option('driver', 'com.mysql.jdbc.Driver').\
            option('user', 'root').\
            option('password', 'jepcpvmj1671E').\
            option('query', query).load()
    df_mysql = df_mysql.withColumnRenamed('InventoryKey', 'inventorykey').withColumnRenamed('FullDateLabel', 'fulldatelabel')\
    .withColumnRenamed('StoreType', 'storetype').withColumnRenamed('ProductName', 'productname').withColumnRenamed('CurrencyName', 'currencyname')\
    .withColumnRenamed('OnHandQuantity', 'onhandquantity').withColumnRenamed('OnOrderQuantity', 'onorderquantity')\
    .withColumnRenamed('SafetyStockQuantity', 'safetystockquantity').withColumnRenamed('UnitCost', 'unitcost')\
    .withColumnRenamed('DaysInStock', 'daysinstock').withColumnRenamed('MaxDayInStock', 'maxdayinstock').withColumnRenamed('Aging', 'aging')
    df_mysql.write\
    .format("org.apache.spark.sql.cassandra")\
    .mode('append')\
    .options(table="tabla3", keyspace="tfm_jhoselinv")\
    .save()

    # Actualizar el desplazamiento para la siguiente parte
    offset += tamaño_parte
    print(offset)

# Detiene la toma de tiempo
fin_tiempo = time.time()

# Calcula el tiempo de ejecución
tiempo_ejecucion = fin_tiempo - inicio_tiempo

print(f"Tiempo de ejecución: {tiempo_ejecucion} segundos")

400000
800000
1200000
1600000
2000000
2400000
2800000
3200000
3600000
4000000
4400000
4800000
5200000
5600000
6000000
6400000
6800000
7200000
7600000
8000000
8013099
Tiempo de ejecución: 447.4105124473572 segundos


In [21]:
df_cassandra = spark.read\
                .format("org.apache.spark.sql.cassandra")\
                .options(table="tabla2", keyspace="tfm_jhoselinv")\
                .load()
# Obtener el número de registros dela consulta realizada a Cassandra
num_rows = df_cassandra.count()
# Imprimir el número de registros
print("Número de registros en Cassandra:", num_rows)

Número de registros en Cassandra: 3406089


### Consulta 3 MongoDB

In [22]:
mongo_ip = "mongodb://localhost:27017/tfm_jhoselinv."
import time 
# se define el tamano de parte
tamaño_parte = 400000
offset = 0

# Inicia la toma de tiempo
inicio_tiempo = time.time()

# Realiza la extracción en partes, recorriendo hasta realizar la migracion completa
while offset < total_registros:
    query = f"SELECT e1.StoreType AS StoreType,TP.InventoryKey,e2.FullDateLabel AS FullDateLabel,\
        e3.ProductName AS ProductName,e4.CurrencyName AS CurrencyName,TP.OnHandQuantity, TP.OnOrderQuantity,\
        TP.SafetyStockQuantity, TP.UnitCost, TP.DaysInStock, TP.MaxDayInStock, TP.Aging FROM factinventory TP\
        LEFT JOIN dimstore e1 ON TP.StoreKey = e1.StoreKey\
        LEFT JOIN dimdate e2 ON TP.DateKey = e2.DateKey\
        LEFT JOIN dimproduct e3 ON TP.ProductKey = e3.ProductKey\
        LEFT JOIN dimcurrency e4 ON TP.CurrencyKey = e4.CurrencyKey  LIMIT {tamaño_parte} OFFSET {offset}"
    # Si es la última parte, ajusta el tamano de parte
    if offset + tamaño_parte >= total_registros:
        tamaño_parte = total_registros - offset
    df_mysql = spark.read.format('jdbc').\
            option('url', 'jdbc:mysql://localhost:3306/spanish').\
            option('driver', 'com.mysql.jdbc.Driver').\
            option('user', 'root').\
            option('password', 'jepcpvmj1671E').\
            option('query', query).load()
    df_mysql = df_mysql.withColumnRenamed('InventoryKey', 'inventorykey').withColumnRenamed('FullDateLabel', 'fulldatelabel')\
    .withColumnRenamed('StoreType', 'storetype').withColumnRenamed('ProductName', 'productname').withColumnRenamed('CurrencyName', 'currencyname')\
    .withColumnRenamed('OnHandQuantity', 'onhandquantity').withColumnRenamed('OnOrderQuantity', 'onorderquantity')\
    .withColumnRenamed('SafetyStockQuantity', 'safetystockquantity').withColumnRenamed('UnitCost', 'unitcost')\
    .withColumnRenamed('DaysInStock', 'daysinstock').withColumnRenamed('MaxDayInStock', 'maxdayinstock').withColumnRenamed('Aging', 'aging')
    df_mysql.write\
    .format("com.mongodb.spark.sql.DefaultSource")\
    .option("uri", mongo_ip + "coleccion3")\
    .mode("append")\
    .save()

    # Actualizar el desplazamiento para la siguiente parte
    offset += tamaño_parte
    print(offset)

# Detiene la toma de tiempo
fin_tiempo = time.time()

# Calcula el tiempo de ejecución
tiempo_ejecucion = fin_tiempo - inicio_tiempo

print(f"Tiempo de ejecución: {tiempo_ejecucion} segundos")

400000
800000
1200000
1600000
2000000
2400000
2800000
3200000
3600000
4000000
4400000
4800000
5200000
5600000
6000000
6400000
6800000
7200000
7600000
8000000
8013099
Tiempo de ejecución: 393.1144061088562 segundos


In [23]:
mongo_ip = "mongodb://localhost:27017/tfm_jhoselinv."
num_documentos = spark.read.format("com.mongodb.spark.sql.DefaultSource").option('uri',mongo_ip+"coleccion3").load()
num_documentos = num_documentos.count()

# Imprimir el número de documentos
print("Número de documentos:", num_documentos)

Número de documentos: 8013099
